In [57]:
# imports 

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [58]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")



API key found and looks good so far!


In [59]:
MODEL = "llama3.2"

In [60]:
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [61]:
web = Website("https://www.t-mobile.com/cell-phone-plans?INTNAV=tNav%3APlans%3AMagenta")
##print(web.text)

In [62]:
system_prompt = (
    "You are a tool designed to analyze websites and provide detailed information "
    "about the prices and benefits of each plan mentioned on the site."
)


In [68]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt = (
            f"You are looking at a website titled '{website.title}'.\n"
            "The contents of this website are as follows:\n"
            "Please provide a short summary of this website in markdown format.\n"
            "Additionally, if applicable, please include the mobile plans offered (AVOID Puerto Rico plan), their prices with autopay, and the number of lines available.\n\n"
            f"{website.text}"
        )
    return user_prompt

In [69]:
# See how this function creates exactly the format above
def messages_for(website):
    return [
        { "role": "system", "content": system_prompt},
        { "role": "user", "content": user_prompt_for(website)}
    ]

In [70]:
# And now: call the OpenAI API. You will get very familiar with this!

def mobilePlans(url):
    website = Website(url)
    response = ollama.chat(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response['message']['content']

In [71]:
# A function to display this nicely in the Jupyter output, using markdown

def displayMobilePlans(url):
    # Generate the combined summary for both websites
    combined_summary = summarize(url)
    
    # Display the combined summary in Markdown format
    display(Markdown(combined_summary))
    

In [73]:
displayMobilePlans("https://www.t-mobile.com/cell-phone-plans?INTNAV=tNav%3APlans%3AMagenta")

# Summary of "Our Best Unlimited Data Cell Phone Plans: Compare Go5G Plan Deals"

The T-Mobile website provides a comprehensive overview of various unlimited data cell phone plans, highlighting the competitive pricing and benefits compared to other carriers like AT&T and Verizon. The plans are designed for both new and existing customers, with specific offerings for families, military members, veterans, and seniors. Key features include unlimited talk, text, and data, with additional perks like streaming services and device upgrade options. All plans are contract-free, ensuring flexibility for users.

## Mobile Plans Offered (Excluding Puerto Rico Plan)

### 1. **Essentials Saver**
- **1 Line:** $55/month 
- **2 Lines:** $90/month 
- **3 Lines:** $115/month 
- **4 Lines:** - 
- **5 Lines:** - 

### 2. **Go5G Next**
- **1 Line:** $105/month 
- **2 Lines:** $180/month 
- **3 Lines:** $230/month 
- **4 Lines:** $280/month 
- **5 Lines:** $330/month 
- **6-8 Lines:** $50/line 
- **9-12 Lines:** $55/line 

### 3. **Go5G Next First Responder**
- **1 Line:** $90/month 
- **2 Lines:** $140/month 
- **3 Lines:** $180/month 
- **4 Lines:** $220/month 
- **5 Lines:** $260/month 
- **6 Lines:** $300/month 
- **7-12 Lines:** $50/line 

### 4. **Go5G Next Military**
- **1 Line:** $90/month 
- **2 Lines:** $140/month 
- **3 Lines:** $180/month 
- **4 Lines:** $220/month 
- **5 Lines:** $260/month 
- **6 Lines:** $300/month 
- **7-12 Lines:** $50/line 

### 5. **Go5G Next 55**
- **1 Line:** $90/month 
- **2 Lines:** $140/month 
- **3 Lines:** $210/month 
- **4 Lines:** $280/month 

### 6. **Magenta MAX**
- **1 Line:** $90/month 
- **2 Lines:** $150/month 
- **3 Lines:** $185/month 
- **4 Lines:** $220/month 
- **5 Lines:** $255/month 
- **6-8 Lines:** $35/line 
- **9-12 Lines:** $40/line 

### 7. **Magenta MAX Military**
- **1 Line:** $75/month 
- **2 Lines:** $110/month 
- **3 Lines:** $135/month 
- **4 Lines:** $160/month 
- **5 Lines:** $185/month 
- **6 Lines:** $210/month 

### 8. **Magenta 55+**
- **1 Line:** $70/month 
- **2 Lines:** $100/month 
- **3 Lines:** $150/month 
- **4 Lines:** $200/month 

### 9. **Magenta First Responder**
- **1 Line:** $60/month 
- **2 Lines:** $90/month 
- **3 Lines:** $105/month 
- **4 Lines:** $120/month 
- **5 Lines:** $135/month 

### 10. **Essentials 55+**
- **1 Line:** $45/month 
- **2 Lines:** $65/month 

### 11. **Go5G Plus**
- **1 Line:** $95/month 
- **2 Lines:** $160/month 
- **3 Lines:** $200/month 
- **4 Lines:** $240/month 

### 12. **Go5G Plus Military**
- **1 Line:** $80/month 
- **2 Lines:** $120/month 
- **3 Lines:** $150/month 
- **4 Lines:** $180/month 

Each plan typically includes unlimited data, 50GB to 100GB of premium data, specific streaming perks, and is free from annual contracts. Additional fees and terms apply, which include a device connection charge of $35/line and variable local surcharges.